In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import os

In [32]:
input_file = 'D:/Calvin/Documents/GTOMSA/Applied Analytics Practicum - CSE 6748/final_data/generic_data_final.csv'


def load_dataset(file_path):
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        print(f"Dataset '{file_path}' loaded successfully.\n")
        return df
    else:
        raise FileNotFoundError(f"The file '{file_path}' does not exist. Please check the file path.")

# Load the dataset
df = load_dataset(input_file)


Dataset 'D:/Calvin/Documents/GTOMSA/Applied Analytics Practicum - CSE 6748/final_data/generic_data_final.csv' loaded successfully.



In [33]:
# Display the first five rows of the dataset
print("First five rows of the dataset:")
display(df.head())


First five rows of the dataset:


,client,sustainer,gender,client_state_rank,all_state_rank,client_zip_rank,all_zip_rank,age,gift_min_t12m,gift_max_t12m,...,drtv_t12m,other_t12m,no_channel_t12m,member_t12m,gift_min_lt,gift_max_lt,gift_avg_lt,gift_count_lt,promo_count_lt,cons_month_lt
0,advcy,0,0,36,29,3553,34364,NaN,1.0,2.0,...,0,0,0,0,1.0,2.0,1.5,2,2,2
1,advcy,0,3,33,41,2,22870,NaN,2.0,2.0,...,0,0,0,0,2.0,2.0,2.0,1,1,1
2,advcy,0,0,42,31,2594,35599,NaN,2.0,2.0,...,0,0,0,0,2.0,2.0,2.0,1,1,1
3,advcy,0,0,42,31,1394,29773,NaN,2.0,2.0,...,0,0,0,0,2.0,2.0,2.0,1,1,1
4,advcy,0,0,42,31,581,28865,NaN,2.0,2.0,...,0,0,0,0,2.0,2.0,2.0,1,1,1


In [34]:
# Define the list of columns you want to drop
columns_to_drop = ['gift_date','account_id']

#dropped because 67.5% of age is missing, safer to just remove this feature
# no_channel and member removed as they all have only 1 value

print(f"Columns intended to drop: {columns_to_drop}\n")

existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]

if existing_columns_to_drop:
    print(f"Columns to be dropped (exist in DataFrame): {existing_columns_to_drop}\n")
else:
    print("No specified columns found in the DataFrame to drop.\n")
df = df.drop(columns=existing_columns_to_drop)
print(f"Successfully dropped columns: {existing_columns_to_drop}\n")

print("Remaining columns after dropping:")
print(df.columns.tolist())


Columns intended to drop: ['gift_date', 'account_id']

No specified columns found in the DataFrame to drop.

Successfully dropped columns: []

Remaining columns after dropping:
['client', 'sustainer', 'gender', 'client_state_rank', 'all_state_rank', 'client_zip_rank', 'all_zip_rank', 'age', 'gift_min_t12m', 'gift_max_t12m', 'gift_avg_t12m', 'gift_count_t12m', 'promo_count_t12m', 'cons_month_t12m', 'dm_t12m', 'tm_t12m', 'wm_t12m', 'digital_t12m', 'f2f_t12m', 'events_t12m', 'drtv_t12m', 'other_t12m', 'no_channel_t12m', 'member_t12m', 'gift_min_lt', 'gift_max_lt', 'gift_avg_lt', 'gift_count_lt', 'promo_count_lt', 'cons_month_lt']


In [35]:

def handle_missing_values(df, strategy='fill', fill_value=0):
    if strategy == 'fill':
        df_filled = df.fillna(fill_value)
        print(f"Missing values filled with {fill_value}.\n")
        return df_filled
    elif strategy == 'drop':
        df_dropped = df.dropna()
        print("Rows with missing values have been dropped.\n")
        return df_dropped
    else:
        raise ValueError("Unsupported strategy for handling missing values.")

# Handle missing values by filling with -1
df = handle_missing_values(df, strategy='fill', fill_value=-1)

Missing values filled with -1.



In [36]:
#remove_flags = ['dm_t12m','tm_t12m','wm_t12m','digital_t12m','f2f_t12m','events_t12m','drtv_t12m','other_t12m','no_channel_t12m','member_t12m','age','sustainer']
remove_flags = ['dm_t12m','tm_t12m','wm_t12m','digital_t12m','f2f_t12m','events_t12m','drtv_t12m','other_t12m','no_channel_t12m','member_t12m','age','sustainer']
numerical_columns = df.select_dtypes(include=[np.number]).drop(columns=remove_flags).columns
print("Numerical columns:", numerical_columns.tolist())

Numerical columns: ['gender', 'client_state_rank', 'all_state_rank', 'client_zip_rank', 'all_zip_rank', 'gift_min_t12m', 'gift_max_t12m', 'gift_avg_t12m', 'gift_count_t12m', 'promo_count_t12m', 'cons_month_t12m', 'gift_min_lt', 'gift_max_lt', 'gift_avg_lt', 'gift_count_lt', 'promo_count_lt', 'cons_month_lt']


Detecting Outliers We'll detect outliers using two methods:

Z-Score Method, Interquartile Range (IQR) Method

In [37]:
# Function to detect outliers using Z-Score
def detect_outliers_zscore(df, numerical_columns, threshold=3):
    outliers = {}
    print("Detecting outliers using Z-Score method\n")
    for column in numerical_columns:
        if df[column].nunique() > 1:
            z_scores = np.abs(stats.zscore(df[column]))
            outliers_indices = np.where(z_scores > threshold)[0]
            outliers[column] = outliers_indices
            print(f"Z-Score Outliers in '{column}': {len(outliers_indices)} detected")
        else:
            outliers[column] = np.array([])
            print(f"Z-Score Outliers in '{column}': 0 detected (constant column)")
    print("\n")
    return outliers

# Detect outliers using Z-Score
outliers_zscore = detect_outliers_zscore(df, numerical_columns)


Detecting outliers using Z-Score method

Z-Score Outliers in 'gender': 0 detected
Z-Score Outliers in 'client_state_rank': 7560 detected
Z-Score Outliers in 'all_state_rank': 5594 detected
Z-Score Outliers in 'client_zip_rank': 0 detected
Z-Score Outliers in 'all_zip_rank': 25369 detected
Z-Score Outliers in 'gift_min_t12m': 937 detected
Z-Score Outliers in 'gift_max_t12m': 506 detected
Z-Score Outliers in 'gift_avg_t12m': 668 detected
Z-Score Outliers in 'gift_count_t12m': 11174 detected
Z-Score Outliers in 'promo_count_t12m': 13281 detected
Z-Score Outliers in 'cons_month_t12m': 21113 detected
Z-Score Outliers in 'gift_min_lt': 371 detected
Z-Score Outliers in 'gift_max_lt': 757 detected
Z-Score Outliers in 'gift_avg_lt': 357 detected
Z-Score Outliers in 'gift_count_lt': 17955 detected
Z-Score Outliers in 'promo_count_lt': 23298 detected
Z-Score Outliers in 'cons_month_lt': 16900 detected




In [38]:
# Function to detect outliers using IQR
def detect_outliers_iqr(df, numerical_columns):
    outliers = {}
    print("Detecting outliers using IQR method...\n")
    for column in numerical_columns:
        Q1 = df[column].quantile(0.10)
        Q3 = df[column].quantile(0.90)
        IQR = Q3 - Q1
        lower_bound = Q1 - 3 * IQR
        upper_bound = Q3 + 3 * IQR
        condition = (df[column] < lower_bound) | (df[column] > upper_bound)
        outliers_indices = df[condition].index
        outliers[column] = outliers_indices
        print(f"IQR Outliers in '{column}': {len(outliers_indices)} detected")
    print("\n")
    return outliers

# Detect outliers using IQR
outliers_iqr = detect_outliers_iqr(df, numerical_columns)

Detecting outliers using IQR method...

IQR Outliers in 'gender': 0 detected
IQR Outliers in 'client_state_rank': 0 detected
IQR Outliers in 'all_state_rank': 0 detected
IQR Outliers in 'client_zip_rank': 0 detected
IQR Outliers in 'all_zip_rank': 0 detected
IQR Outliers in 'gift_min_t12m': 15541 detected
IQR Outliers in 'gift_max_t12m': 17452 detected
IQR Outliers in 'gift_avg_t12m': 21163 detected
IQR Outliers in 'gift_count_t12m': 24337 detected
IQR Outliers in 'promo_count_t12m': 129 detected
IQR Outliers in 'cons_month_t12m': 19038 detected
IQR Outliers in 'gift_min_lt': 11999 detected
IQR Outliers in 'gift_max_lt': 25544 detected
IQR Outliers in 'gift_avg_lt': 15020 detected
IQR Outliers in 'gift_count_lt': 2773 detected
IQR Outliers in 'promo_count_lt': 172 detected
IQR Outliers in 'cons_month_lt': 3495 detected




Removing Outliers Based on IQR We'll remove the detected outliers using the IQR method to obtain a dataset without outliers

In [39]:
def remove_outliers_iqr(df, numerical_columns):
    print("Removing outliers based on IQR method...\n")
    Q1 = df[numerical_columns].quantile(0.10)
    Q3 = df[numerical_columns].quantile(0.90)
    IQR = Q3 - Q1
    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR
    condition = ~((df[numerical_columns] < lower_bound) | (df[numerical_columns] > upper_bound)).any(axis=1)
    df_no_outliers = df[condition]
    print("Outliers removed.")
    print(f"Original dataset size: {df.shape[0]} rows")
    print(f"Dataset without outliers size: {df_no_outliers.shape[0]} rows\n")
    return df_no_outliers

# Remove outliers using IQR
df_without_outliers = remove_outliers_iqr(df, numerical_columns)


Removing outliers based on IQR method...

Outliers removed.
Original dataset size: 849579 rows
Dataset without outliers size: 767022 rows



In [40]:

def display_dataset_without_outliers(df):
    print("Displaying the dataset without outliers:\n")
    # Display the first 5 rows
    print("First five rows of the dataset without outliers:")
    display(df.head())
    
    print("\nSummary Statistics of Dataset Without Outliers:")
    display(df.describe())
    
    # Display the number of rows and columns
    print(f"\nDataset without outliers contains {df.shape[0]} rows and {df.shape[1]} columns.\n")

display_dataset_without_outliers(df_without_outliers)


Displaying the dataset without outliers:

First five rows of the dataset without outliers:


,client,sustainer,gender,client_state_rank,all_state_rank,client_zip_rank,all_zip_rank,age,gift_min_t12m,gift_max_t12m,...,drtv_t12m,other_t12m,no_channel_t12m,member_t12m,gift_min_lt,gift_max_lt,gift_avg_lt,gift_count_lt,promo_count_lt,cons_month_lt
0,advcy,0,0,36,29,3553,34364,-1.0,1.0,2.0,...,0,0,0,0,1.0,2.0,1.5,2,2,2
1,advcy,0,3,33,41,2,22870,-1.0,2.0,2.0,...,0,0,0,0,2.0,2.0,2.0,1,1,1
2,advcy,0,0,42,31,2594,35599,-1.0,2.0,2.0,...,0,0,0,0,2.0,2.0,2.0,1,1,1
3,advcy,0,0,42,31,1394,29773,-1.0,2.0,2.0,...,0,0,0,0,2.0,2.0,2.0,1,1,1
4,advcy,0,0,42,31,581,28865,-1.0,2.0,2.0,...,0,0,0,0,2.0,2.0,2.0,1,1,1



Summary Statistics of Dataset Without Outliers:


,sustainer,gender,client_state_rank,all_state_rank,client_zip_rank,all_zip_rank,age,gift_min_t12m,gift_max_t12m,gift_avg_t12m,...,drtv_t12m,other_t12m,no_channel_t12m,member_t12m,gift_min_lt,gift_max_lt,gift_avg_lt,gift_count_lt,promo_count_lt,cons_month_lt
count,767022.000000,767022.000000,767022.000000,767022.000000,767022.000000,767022.000000,767022.000000,767022.000000,767022.000000,767022.000000,...,767022.000000,767022.000000,767022.000000,767022.000000,767022.000000,767022.000000,767022.000000,767022.000000,767022.000000,767022.000000
mean,0.557062,1.058642,39.504002,38.757668,20046.122083,31561.875994,41.964163,32.122568,52.867713,39.777560,...,0.008468,0.087718,0.001289,0.030021,17.664065,76.444478,33.372553,48.162793,32.640876,9.425469
std,0.496734,1.208877,11.399501,11.599942,7206.408250,6695.251784,39.539953,38.214317,68.662189,45.264907,...,0.091630,0.282885,0.035885,0.170646,21.377946,100.224244,34.109798,69.431021,50.045599,15.121403
min,0.000000,0.000000,1.000000,1.000000,2.000000,9.000000,-30.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
25%,0.000000,0.000000,33.000000,31.000000,17027.000000,29950.000000,-1.000000,10.000000,19.000000,15.000000,...,0.000000,0.000000,0.000000,0.000000,5.000000,20.000000,13.260000,4.000000,2.000000,1.000000
50%,1.000000,1.000000,43.000000,43.000000,21701.000000,33720.000000,58.000000,20.000000,25.000000,25.000000,...,0.000000,0.000000,0.000000,0.000000,10.000000,40.000000,22.270000,20.000000,12.000000,3.000000
75%,1.000000,3.000000,49.000000,49.000000,25780.000000,35962.000000,79.000000,35.000000,52.599998,50.000000,...,0.000000,0.000000,0.000000,0.000000,20.000000,100.000000,39.090000,64.000000,41.000000,11.000000
max,1.000000,3.000000,51.000000,51.000000,28674.000000,37517.000000,129.000000,382.000000,601.010010,389.350000,...,1.000000,1.000000,1.000000,1.000000,190.000000,955.336035,361.310000,645.000000,396.000000,137.000000



Dataset without outliers contains 767022 rows and 30 columns.



In [41]:
def save_dataset_without_outliers(df, output_path):
    try:
        df.to_csv(output_path, index=False)
        print(f"Dataset without outliers saved to '{output_path}'.\n")
    except Exception as e:
        print(f"Error saving file: {e}\n")


output_file = 'D:/Calvin/Documents/GTOMSA/Applied Analytics Practicum - CSE 6748/final_data/genericdata_without_outliers.csv'

# Save the cleaned dataset
save_dataset_without_outliers(df_without_outliers, output_file)


Dataset without outliers saved to 'D:/Calvin/Documents/GTOMSA/Applied Analytics Practicum - CSE 6748/final_data/genericdata_without_outliers.csv'.

